## Setup
- load train and test datasets, create TensorFlow Datasets from them
- define main architecutre params

In [1]:
%load_ext autoreload
%autoreload 2

from __future__ import absolute_import, division, print_function

import logging
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import skimage

from src.data_utils import Dataset
from src.network import unet_3d

# tf.enable_eager_execution()

# get TF logger - set it to info for more tracking process
log = logging.getLogger('tensorflow')
log.setLevel(logging.DEBUG)

In [2]:
batch_size = 2
train_dataset = Dataset.load_dataset(
    '../data/processed/train_dataset.pckl'
).create_tf_dataset().shuffle(50).repeat().padded_batch(
    batch_size=batch_size,
    padded_shapes=([None, 128, 128, 1], [None, 128, 128, 3]))
test_dataset = Dataset.load_dataset(
    '../data/processed/test_dataset.pckl'
).create_tf_dataset().shuffle(50).repeat().padded_batch(
    batch_size=batch_size,
    padded_shapes=([None, 128, 128, 1], [None, 128, 128, 3]))

# setup dataset iterator objects, idea from:
# http://adventuresinmachinelearning.com/tensorflow-dataset-tutorial/
iterator = tf.data.Iterator.from_structure(
    train_dataset.output_types,
    train_dataset.output_shapes
)
training_init_op = iterator.make_initializer(train_dataset)
test_init_op = iterator.make_initializer(test_dataset)
next_element = iterator.get_next()

In [3]:
# create the neural network model
train_phase = tf.placeholder(tf.bool, name="is_training")
logits = unet_3d(next_element[0], training=train_phase, depth=3, n_base_filters=16)

# add the optimizer and weigthed softmax loss
# see https://stackoverflow.com/a/44563055
class_weights = tf.cast(tf.constant([[0.00408978, 0.70013423, 0.295776]]), tf.float32)
# deduce weights for batch samples based on their true label
class_weights = tf.reduce_sum(tf.cast(next_element[0], tf.float32) * class_weights, axis=-1)
loss = tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=next_element[1], weights=class_weights)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = tf.train.AdamOptimizer().minimize(loss)
    
# get accuracy
prediction = tf.argmax(logits, axis=-1)
labels = tf.argmax(next_element[1], -1)
equality = tf.equal(prediction, labels)
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))

# get mean iou
iou, conf_mat = tf.metrics.mean_iou(
    labels=labels, 
    predictions=tf.cast(prediction, tf.int32),
    num_classes=3
)

# setup variables for training
train_n = 70
test_n = 10
epochs = 100

# run the training
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
sess.run(training_init_op)
saver = tf.train.Saver()

for i in range(epochs * train_n):
    # how to run tf.metrics, idea taken from: https://stackoverflow.com/a/46414395
    l, _, acc, _, _ = sess.run([loss, train_op, accuracy, iou, conf_mat], feed_dict={train_phase:True})
    miou = sess.run([iou])
    if i % 50 == 0:
        print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i, l, acc * 100))
        print("Epoch: {}, loss: {:.3f}, training IOU: {:.2f}%".format(i, l, miou[0] * 100))


# re-initialize the iterator, but this time with test data
sess.run(test_init_op)
test_iters = epochs * test_n
avg_acc = 0
avg_miou = 0
for i in range(test_iters):
    acc, miou = sess.run([accuracy, iou], feed_dict={train_phase:False})
    avg_acc += acc
    avg_miou += miou
print("Average test set accuracy over {} iterations is {:.2f}%".format(test_iters, (avg_acc / test_iters) * 100))
print("Average test set IOU over {} iterations is {:.2f}%".format(test_iters, (avg_miou / test_iters) * 100))

save_path = saver.save(sess, "/tmp/model.ckpt")
print("Model saved in path: %s" % save_path)

DEBUG:tensorflow:conv1: (?, ?, 128, 128, 16)
DEBUG:tensorflow:conv2: (?, ?, 128, 128, 32)
DEBUG:tensorflow:maxpool layer: (?, ?, 64, 64, 32)
DEBUG:tensorflow:conv1: (?, ?, 64, 64, 32)
DEBUG:tensorflow:conv2: (?, ?, 64, 64, 64)
DEBUG:tensorflow:maxpool layer: (?, ?, 32, 32, 64)
DEBUG:tensorflow:conv1: (?, ?, 32, 32, 64)
DEBUG:tensorflow:conv2: (?, ?, 32, 32, 128)
DEBUG:tensorflow:upconv layer: (?, ?, 64, 64, 128)
DEBUG:tensorflow:concat layer: (?, ?, 64, 64, 64)
DEBUG:tensorflow:up_conv layer1: (?, ?, 64, 64, 64)
DEBUG:tensorflow:up_conv layer2 : (?, ?, 64, 64, 64)
DEBUG:tensorflow:upconv layer: (?, ?, 128, 128, 64)
DEBUG:tensorflow:concat layer: (?, ?, 128, 128, 32)
DEBUG:tensorflow:up_conv layer1: (?, ?, 128, 128, 32)
DEBUG:tensorflow:up_conv layer2 : (?, ?, 128, 128, 32)
DEBUG:tensorflow:output layer:: (?, ?, 128, 128, 3)


ResourceExhaustedError: OOM when allocating tensor with shape[2,96,32,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node synthesis_conv3d_0/Conv3D (defined at /home/daniel/Desktop/deepmind_unet/src/network.py:157)  = Conv3D[T=DT_FLOAT, data_format="NDHWC", dilations=[1, 1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](synthesis_concat_0, synthesis_conv3d_0/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node softmax_cross_entropy_loss/assert_broadcastable/AssertGuard/Assert/Switch/_65}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_975_s...ert/Switch", tensor_type=DT_BOOL, _device="/job:localhost/rep